# Registration tools

This notebook is part of [blabla citation papier].
Before executing it, please make sure you have installed the tools properly (see installation part).

Please run the cells one by one, provide input when it is asked and press enter to validate your input, but do not modify the content of the cells.

The data structure required is the following : one main folder with the raw movies (3D stacks in time), which can be multichannel or not.
The first part prepares the data, cut the movie into a timesequence and asks for the parameters. The second part does the actual registration.
The process can take few minutes depending on the size of your data. If there is an error, it will be printed either below or in the terminal window. To solve it, please consider the 'Troubleshooting' section.


1- Importing packages (no input required)

In [1]:
import tifffile
import os
import registrationtools
#from registrationtools import utils
import json
import numpy as np
from glob import glob
from ensure import check ##need to pip install !
from pathlib import Path

C:\Users\gros\Anaconda3\envs\registration\lib\site-packages\IO\IO.py:31: UserWarning: KLB library is not installed
  warnings.warn("KLB library is not installed")


pyklb library not found, klb files will not be generated


In [16]:
def dimensions(list_paths:list) :
    # Take the path to an image and returns it size in the dimensions CTZXY
    movie = tifffile.imread(list_paths[0])
    name_movie = Path(list_paths[0]).stem
    dim_correct = False
    while dim_correct == False :
        print('\nThe dimensions of ',name_movie, 'are ',movie.shape,'.')
        dimensions=input('What is the order of the dimensions (for example TZCYX or XYZT) ? ')
        sorted_axes = sorted(dimensions)
        if ''.join(sorted_axes) == "CTXYZ":
            number_channels = movie.shape[dimensions.find('C')]
            dim_correct=True
        elif ''.join(sorted_axes) == "TXYZ":
            number_channels = 1
            dim_correct=True
        elif len(sorted_axes) != len(movie.shape) :
            print('Error : Number of dimensions is indim_correct')
            dim_correct = False 
        else:
            print('The letters you choose has to be among these letters : X,Y,Z,C,T, with XYZT mandatory, and no other letters are allowed. \nEvery letter can to be included only once.')

        number_timepoints = movie.shape[dimensions.find('T')]
        if 'C' in dimensions :
            number_channels = movie.shape[dimensions.find('C')]
        else :
            number_channels = 1
        if dim_correct == True :
            depth = movie.shape[dimensions.find('Z')]
            size_X = movie.shape[dimensions.find('X')]
            size_Y = movie.shape[dimensions.find('Y')]

            print('So',name_movie, 'has',number_channels,'channels, ',number_timepoints,'timepoints, the depth in z is',depth,'pixels and the XY plane measures ',size_X,'x',size_Y,'pixels')
            dim_correct = int(input('Correct ? (1 for yes, 0 for no)'))
    for path in list_paths : #checking if the movies have the same dimensions in C and T, otherwise the registration cannot be computed.
        mov = tifffile.imread(path)
        check(mov.shape[dimensions.find('T')]).equals(number_timepoints).or_raise(Exception, 'These movies do not all have the same number of timepoints.')
        check(mov.shape[dimensions.find('C')]).equals(number_channels).or_raise(Exception, 'These movies do not all have the same number of channels.')
    return(number_channels,number_timepoints,depth,size_X,size_Y)

def sort_by_channels_and_timepoints(list_paths:str,channels:str,number_timepoints:int):
#Takes a list of movies and their dimension in (T,C) and cut into timesequence of stakcs. Works for one channel or multiple channels images.
#returns a list of the directories name where the regitration should happen
    list_directories=[]
    if len(channels) == 1:
        for path in list_paths :
            name_movie = Path(path).stem
            directory=name_movie+'_'+channels[0]
            cut_timesequence(path_to_movie =path,
                             directory=directory,
                             current_channel=channels[0],
                             number_timepoints = number_timepoints,
                             number_channels = len(channels))
            list_directories.append(directory)


    else : #if multichannels
        for n in range(len(channels)) :
            for path in list_paths :
                name_movie = Path(path).stem
                directory = name_movie+'_'+channels[n]
                cut_timesequence(path_to_movie = path,
                                 directory=directory,
                                 current_channel=n,
                                 number_timepoints = number_timepoints,
                                 number_channels = len(channels))
                list_directories.append(directory)
    return(list_directories)


def cut_timesequence (path_to_movie:str, directory:str, current_channel:int, number_timepoints:int,number_channels:int):
    #take a movie and cut it into a timesequence in the given directory
    movie = tifffile.imread(path_to_movie)
    position_t = np.argwhere(np.array(movie.shape)==number_timepoints)[0][0]
    movie=np.moveaxis(movie,source=position_t,destination=0)
    path_to_data=os.path.dirname(path_to_movie)
    os.mkdir(os.path.join(path_to_data,directory))
    os.mkdir(os.path.join(path_to_data+'/'+directory,"trsf"))
    os.mkdir(os.path.join(path_to_data+'/'+directory,"output"))
    os.mkdir(os.path.join(path_to_data+'/'+directory,"proj_output"))
    os.mkdir(os.path.join(path_to_data+'/'+directory,"stackseq"))
    if number_channels >1 :
        position_c = np.argwhere(np.array(movie.shape)==number_channels)[0][0]
        movie=np.moveaxis(movie,source=position_c,destination=2)
        for t in range(number_timepoints) :
            stack = movie[t,:,current_channel,:,:]
            tifffile.imwrite(path_to_data+'/'+directory+"/stackseq/movie_t"+str(format(t,'03d')+'.tif'),stack)
    else : 
        for t in range(number_timepoints) :
            stack = movie[t,:,:,:]
            tifffile.imwrite(path_to_data+'/'+directory+"/stackseq/movie_t"+str(format(t,'03d')+'.tif'),stack)

def reference_channel(channels:list) :
#Ask for the reference channel among the channels given (with safety check)
    if len(channels) == 1 :
        ch_ref=channels[0]
    else :
        channel_correct=False
        print('\n Among the channels'+ str(channels)+', you need a reference channel to compute the registration. A good option is generally a marker that is uniformly expressed')
        while channel_correct==False :
            ch_ref = input('Name of the reference channel :')
            if ch_ref not in channels :
                print('The reference channel is not in', channels,'(do not put any other character than the name itself)')
                channel_correct=False
            else :
                channel_correct=True
        #modify here the channels to put the reference first (we need the transformation for this channel first)
        #PROBLEM : MIXING THE CHANNELS
        position_chref = np.argwhere(np.array(channels)==ch_ref)[0][0]
        channels.insert(0, channels.pop(position_chref))
    return(channels,ch_ref)

def get_voxel_sizes() :
#ask the user for the voxel size of his input and what voxel size does he want in output. Returns 2 tuples for this values.
    print('\nTo register properly, you need to specify the voxel size of your input image. This can be found in Fiji, Image>Show Info. You can choose to have another voxel size on the registered image , for example to have an isotropic output image (voxel size [1,1,1]), Or you can also choose to keep the same voxel size')
    print('Voxel size of your original image (XYZ successively) :')
    x= float(input('X :'))
    y= float(input('Y :'))
    z= float(input('Z :'))
    voxel_size_input = [x,y,z]
    print('Initial voxel size =',voxel_size_input)
    change_voxel_size = int(input('Do you want to change the voxel size of your movies ? (1 for yes, 0 for no) : '))
    if change_voxel_size==1 :
        print('\nVoxel size of your image after transformation (XYZ):')
        x= float(input('X :'))
        y= float(input('Y :'))
        z= float(input('Z :'))
        voxel_size_output = [x,y,z]
    elif change_voxel_size==0 :
        voxel_size_output = voxel_size_input
    print('\nVoxel size after transformation =',voxel_size_output)
    return(voxel_size_input,voxel_size_output)

def get_paths() :
#Ask the user for the folder containing his data, find the movies is they are in tiff format and put them in a list after confirmation
    correct=False
    while correct==False :
        path_folder = input('Path to the folder of the movie(s) (in tiff format only) : ')
        paths_movies = sorted(glob(rf'{path_folder}/*.tif'))
        print('You have',len(paths_movies),'movie(s), which is (are) :')
        for path_movie in paths_movies :
            print(Path(path_movie).stem)
        correct = int(input('Correct ? (1 for yes, 0 for no)'))
    return (paths_movies)

def get_channels_name(number_channels:int) :
    channels=[]
    if number_channels == 1:
        ch = str(input('Name of the channel : '))
        channels.append(ch)
    else : #if multichannels
        for n in range(number_channels) :
            ch = str(input('Name of channel n°'+str(n+1) +' : '))
            channels.append(ch)
    return(channels)

def get_trsf_type() :
    list_trsf_types = ['rigid2D','rigid3D','translation2D','translation3D']
    print('You can choose to apply different transformation types depending on your data : ',list_trsf_types)
    trsf_correct = False
    while trsf_correct == False :
        trsf_type = str(input('Which one do you want to use ? (please enter the name of the transformation only, no other character)'))
        if trsf_type not in list_trsf_types :
            print(('You can only choose a transformation that is in this list :',list_trsf_types))
        else :
            trsf_correct = True

    return(trsf_type)


In [18]:
list_paths = get_paths()
if os.path.exists(list_paths[0]) == True :
    number_channels,number_timepoints,depth,size_X,size_Y = dimensions(list_paths)  
    channels=get_channels_name(number_channels)
    print(channels)
    channels,ch_ref = reference_channel(channels)
    print(channels)
    list_directories = sort_by_channels_and_timepoints(list_paths = list_paths,
                                    channels = channels,
                                    number_timepoints = number_timepoints)
    
    voxel_size_input, voxel_size_output = get_voxel_sizes()
    trsf_type = get_trsf_type()
else :
    print('File not found at ', list_paths[0]) #pb : path with backslashesand one forward slash at the end

Path to the folder of the movie(s) (in tiff format only) : C:\Users\gros\Desktop\DATA\20230207_fgf\20230329
You have 2 movie(s), which is (are) :
2
4
Correct ? (1 for yes, 0 for no)1

The dimensions of  2 are  (102, 41, 2, 512, 512) .
What is the order of the dimensions (for example TZCYX or XYZT) ? TZCXY
So 2 has 2 channels,  102 timepoints, the depth in z is 41 pixels and the XY plane measures  512 x 512 pixels
Correct ? (1 for yes, 0 for no)1
Name of channel n°1 : sulfo
Name of channel n°2 : h2b
['sulfo', 'h2b']

 Among the channels['sulfo', 'h2b'], you need a reference channel to compute the registration. A good option is generally a marker that is uniformly expressed
Name of the reference channel :h2b
['h2b', 'sulfo']

To register properly, you need to specify the voxel size of your input image. This can be found in Fiji, Image>Show Info. You can choose to have another voxel size on the registered image , for example to have an isotropic output image (voxel size [1,1,1]), Or you c

In [24]:
list_paths = ['C:\\Users\\gros\\Desktop\\DATA\\20230207_fgf\\20230329\\2.tif','C:\\Users\\gros\\Desktop\\DATA\\20230207_fgf\\20230329\\4.tif']

In [23]:
for ind,path in enumerate(list_paths) :
    path_to_data=os.path.dirname(path)
    name_movie = Path(path).stem
    movie = tifffile.imread(path)
    ref_timepoint = int(movie.shape[0]/2)
    for c in channels :
        directory = name_movie+'_'+c
        if c==ch_ref :
            compute_transf = 1
            trsf_folder = rf'{path_to_data}/{directory}/trsf/'
        else :
            compute_transf = 0
            trsf_folder = rf'{path_to_data}/{name_movie}_{ch_ref}/trsf/'
                # # # # ##JSON FILE CONFIG
        data = {
          "path_to_data": rf'{path_to_data}/{directory}/stackseq/',
          "file_name": "movie_t{t:03d}.tif",
          "trsf_folder": trsf_folder,
          "output_format": rf'{path_to_data}/{directory}/output/',
          "projection_path": rf'{path_to_data}/{directory}/proj_output/',
          "check_TP": 0,
          "voxel_size": voxel_size_input,
          "voxel_size_out" : voxel_size_output,
          "first": 0,
          "last": movie.shape[0]-1,
          "not_to_do": [],
          "compute_trsf": compute_transf,
          "ref_TP": ref_timepoint,
          "trsf_type": trsf_type,
          "padding": 1,
          "recompute": 1,

          "apply_trsf":1,
          "out_bdv": "",
          "plot_trsf":0

        }

        json_string=json.dumps(data)
        # with open(path_to_json,'w') as outfile :
        #     outfile.write(json_string)

        tr = registrationtools.TimeRegistration(data)
        tr.run_trsf()


Starting experiment
The registration will run with the following arguments:

---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- File format -----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
path_to_data             : C:\Users\gros\Desktop\DATA\20230207_fgf\20230329/2_h2b/stackseq/
file_name                : movie_t{t:03d}.tif
trsf_folder              : C:\Users\gros\Desktop\DATA\20230207_fgf\20230329/2_h2b/trsf/
output_format

The previous cell saved an image sequence. Here you can save the output as a movie (Because of the transformation, the output can be significantly bigger than the original)

In [25]:
for ind,path in enumerate(list_paths) :
    path_to_data=os.path.dirname(path)
    name_movie = Path(path).stem
    movie = tifffile.imread(path)
    stack0 =tifffile.imread(rf"{path_to_data}/{name_movie+'_'+channels[0]}/output/movie_t000.tif")
    registered_movie = np.zeros((number_timepoints,stack0.shape[0],number_channels,stack0.shape[1],stack0.shape[2]),dtype=np.float16) #one movie per channel, of format (t,z,y,x).Datatype uint16 or float34 is necessary to export as hyperstack
    for ind_c,c in enumerate(channels) :
        directory = name_movie+'_'+c
        for t in range(movie.shape[0]) :
            stack =tifffile.imread(rf"{path_to_data}/{directory}/output/movie_t{format(t,'03d')}.tif")
            #we take each stack in a given timepoint
            registered_movie[t,:,ind_c,:,:]=stack #and put it in a new hyperstack
    tifffile.imwrite(path_to_data+rf"/{name_movie}_registered.tif",registered_movie.astype(np.float16)) #write a hyperstack in the main folder
    print('saved registered', name_movie, 'of size', registered_movie.shape)

saved registered 2 of size (102, 186, 2, 396, 455)
saved registered 4 of size (102, 162, 2, 478, 459)


If you want to save the json file of your registration, fill the line below (this might be useful in case of debugging or safety checks)
example : C:\Users\username\Desktop\Registration\json_files\temporal.json

In [52]:
path_to_json = (input('Where to save the json file :')).replace("\\", "/")
with open(path_to_json,'w') as outfile :
    outfile.write(json_string)